In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import os

### Definindo funções comuns

In [22]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

def extrair_links(html, lista):
    soup = BeautifulSoup(html, 'html.parser')
    produtos = soup.find_all('div', class_='produto')
    
    for produto in produtos:
        link = produto.find('a', href=True)

        s_desc = None
        c_desc = None
        p_desc = None

        try:
            preco_div = produto.find('div', {'data-target': 'preco-produto'})

            if preco_div:
                if preco_div.find('p', {'class': 'text-muted small font-weight-semibold'}):  # Caso com desconto
                    s_desc_elem = preco_div.find('p', {'class': 'text-muted small font-weight-semibold'})
                    c_desc_elem = preco_div.find('span', {'class': 'font-weight-bold font-xl blueText'})
                    s_desc = s_desc_elem.text if s_desc_elem else None
                    c_desc = c_desc_elem.text if c_desc_elem else None

                    p_desc_div = produto.find('div', {'data-target': 'desconto-produto'})
                    p_desc_elem = p_desc_div.find('div', {'class': "desconto-produto font-weight-bold w-100"})
                    p_desc = p_desc_elem.text if p_desc_elem else None

                elif preco_div.find('span', {'class': 'font-xl blueText fontOutfitExtraBold'}):  # Caso sem desconto
                    s_desc_elem = preco_div.find('span', {'class': 'font-xl blueText fontOutfitExtraBold'})
                    s_desc = s_desc_elem.text if s_desc_elem else None
                    c_desc = s_desc

        except AttributeError as e:
            print(f'Erro ao extrair informações do produto {link}: {e}')
            
        resultado = {'URL': 'https://www.farmaciarosario.com.br' + link['href'], 'Preço sem desconto': s_desc, 'Preço com desconto': c_desc, '% de desconto': p_desc}
        
        lista.append(resultado)
    return lista

def extrair_infos(url):
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        nome_elemento = soup.find('h1', {'data-target': 'nome_produto'})
        ean_elemento = soup.find('span', class_='text-muted text-descricao-produto fontOutfitRegular')
        marca_elemento = soup.find('p', class_='text-muted my-2 font-sm font-08rem mb-3')
        descricao_elemento = soup.find('div', {'data-target': 'produto-descricao'})

        nome = nome_elemento.get_text(strip=True) if nome_elemento else None
        ean_text = ean_elemento.get_text(strip=True) if ean_elemento else None
        marca = marca_elemento.find('a').get_text(strip=True) if marca_elemento and marca_elemento.find('a') else None
        descricao = descricao_elemento.get_text(strip=True) if descricao_elemento else None

        ean = ean_text.split('EAN: ')[1] if ean_text else None

        informacoes = {
            'Produto': nome,
            'EAN': ean,
            'Marca': marca,
            'Descrição': descricao,
        }
        return informacoes

    except Exception as e:
        print(f"Erro ao processar a URL {url}: {e}")
        return None
  
def process_url(url):
    info = extrair_infos(url)
    if info:
        return info

A primeira aba do código trabalha na extração dos links e preços dos produtos por categorias, e posteriormente acessamos cada link para extrair o restante das informações.

### Extraindo links

In [17]:
categorias = ['medicamentos', 'sua-beleza', 'manipulacao', 'infantil', 'dermocosmeticos', 'cuidados-pessoais']

driver = webdriver.Chrome()
links = []

for categoria in categorias:
    ref = f"https://www.farmaciarosario.com.br/categorias/{categoria}"
    driver.get(ref)
    wait = WebDriverWait(driver, 10)

    while True:
        try:
            wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, 'div.load.load-body')))
            extrair_links(driver.page_source, links)

            # Encontrar e clicar no botão de próxima página
            next_page_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[data-pagina="next"]')))
            next_page_button.click()
            time.sleep(2)

        except (StaleElementReferenceException, TimeoutException):
            break

driver.quit()

### Extraindo informações

In [18]:
# Criando df com links e salvando arquivo
df_links = pd.DataFrame(links)
df_links = df_links.drop_duplicates()
df_links.to_csv('df_links.csv', index=False)

In [23]:
max_workers = os.cpu_count()

In [25]:
infos = []

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    # Use the map function to apply the process_url function to each URL in parallel
    results = list(tqdm(executor.map(process_url, df_links['URL']), total=len(df_links)))

# Collect the results
infos = list(filter(None, results))

 17%|█▋        | 2075/11930 [11:49<55:40,  2.95it/s]  

Erro ao processar a URL https://www.farmaciarosario.com.br//siblima-72comprimidos: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


 26%|██▋       | 3136/11930 [17:58<43:04,  3.40it/s]   

Erro ao processar a URL https://www.farmaciarosario.com.br//acular-ls-colirio-5ml: HTTPSConnectionPool(host='www.farmaciarosario.com.br', port=443): Max retries exceeded with url: //acular-ls-colirio-5ml (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000269F7437690>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Erro ao processar a URL https://www.farmaciarosario.com.br//atropina-05-colirio-5ml: HTTPSConnectionPool(host='www.farmaciarosario.com.br', port=443): Max retries exceeded with url: //atropina-05-colirio-5ml (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000269F78ED5D0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Erro ao processar a URL https://www.farmaciarosario.com.br//latonan-005mg-5mg-solucao-oftalmica-com-25ml: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Erro ao processar a URL https://www.farmaciar

 38%|███▊      | 4569/11930 [27:29<6:00:12,  2.94s/it]

...

 38%|███▊      | 4570/11930 [27:36<7:51:34,  3.84s/it]

Erro ao processar a URL https://www.farmaciarosario.com.br//reactine-10mg-10capsulas: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


 48%|████▊     | 5726/11930 [33:51<40:39,  2.54it/s]  

Erro ao processar a URL https://www.farmaciarosario.com.br//shampoo-clear-anticaspa-flor-de-cerejeira-200ml: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Erro ao processar a URL https://www.farmaciarosario.com.br//tintura-amend-magnific-color-7-0-louro-natural: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


100%|██████████| 11930/11930 [1:09:33<00:00,  2.86it/s]


### Tratamento de dados para output final

In [26]:
df_infos = pd.DataFrame(infos)
df_infos.to_csv('df_infos.csv')

In [27]:
# Algumas linhas retornaram erros para o dataframe de informações no momento da extração, então essas serão removidas do dataframe de links para termos o mesmo número de linhas
links_a_remover = ['https://www.farmaciarosario.com.br/siblima-72comprimidos', 'https://www.farmaciarosario.com.br/acular-ls-colirio-5ml', 'https://www.farmaciarosario.com.br/atropina-05-colirio-5ml', 'https://www.farmaciarosario.com.br/latonan-005mg-5mg-solucao-oftalmica-com-25ml', 'https://www.farmaciarosario.com.br/tart-brimonidina-solucao-oftalmica-2mg-5ml-510211', 'https://www.farmaciarosario.com.br/salonpas-ice-cold-spray-120ml', 'https://www.farmaciarosario.com.br/reactine-10mg-10capsulas', 'https://www.farmaciarosario.com.br/shampoo-clear-anticaspa-flor-de-cerejeira-200ml', 'https://www.farmaciarosario.com.br/tintura-amend-magnific-color-7-0-louro-natural', ]
df = df_links[~df_links['URL'].isin(links_a_remover)]
df = df.reset_index(drop=True)

# Juntando os dois dataframes
df_final = pd.concat([df_infos, df.iloc[:, -2:]], axis=1)

In [28]:
# Adicionando colunas finais
df_final['Farmacia'] = 'Santo Remedio'
df_final['Região'] = 'Sudeste'
df_final['Cidade'] = 'Sao Paulo'

In [29]:
df_final.to_csv('df_final.csv', index=False)